# Lesson 4: Persistence and Streaming

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

from gigachat.models import Chat, Messages, MessagesRole, chat_completion
from langchain_gigachat.chat_models import GigaChat

_ = load_dotenv(find_dotenv())

api_key  = os.getenv('GIGACHAT_API_KEY')

model = GigaChat(credentials=api_key, verify_ssl_certs=False, temperature = 0)

In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
tool = TavilySearchResults(max_results=2)

In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [7]:
!pip install gigagraph-checkpoint-sqlite

  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.10
    Uninstalling langsmith-0.2.10:
      Successfully uninstalled langsmith-0.2.10


In [24]:
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.memory import MemorySaver

memory = SqliteSaver.from_conn_string(":memory:")
memory = MemorySaver()

In [25]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [26]:
prompt = """
Вы - толковый научный сотрудник. Используйте поисковую систему для поиска информации. \
Вам разрешено совершать несколько звонков (как одновременно, так и последовательно). \
Ищите информацию только тогда, когда вы уверены в том, что вам нужно. \
Если вам нужно найти какую-то информацию, прежде чем задать уточняющий вопрос, вы можете это сделать!
"""
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [27]:
messages = [HumanMessage(content="Какая погода сегодня в Санкт-Петербурге?")]

In [28]:
thread = {"configurable": {"thread_id": "1"}}

In [29]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'погода сегодня в Санкт-Петербурге'}}, 'functions_state_id': 'c4b54682-4e6d-4e62-b7bd-61b782fdd37e'}, response_metadata={'token_usage': {'prompt_tokens': 197, 'completion_tokens': 38, 'total_tokens': 235}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-b23b85db-4933-4431-8561-038b26fe40c4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'погода сегодня в Санкт-Петербурге'}, 'id': '9a5832b3-bad3-4c01-a50d-37d2f0ddf747', 'type': 'tool_call'}])]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'погода сегодня в Санкт-Петербурге'}, 'id': '9a5832b3-bad3-4c01-a50d-37d2f0ddf747', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content="[{'url': 'https://www.meteoservice.ru/weather/today/sankt-peterburg', 'content': 'Санкт-Петербурге сегодня по часам\\nКарточки погоды на сегодня\\nХарактер погоды\\

In [30]:
messages = [HumanMessage(content="Что насчёт Москвы?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'погода сегодня в Москве'}}, 'functions_state_id': '5682dbb9-918c-46cb-bf2f-b432daf17939'}, response_metadata={'token_usage': {'prompt_tokens': 976, 'completion_tokens': 36, 'total_tokens': 1012}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-f10028c0-12bf-4ed7-ae4b-a8d79ca41172-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'погода сегодня в Москве'}, 'id': '76ebf878-ca39-47a7-a7c9-b20cdcfbba2e', 'type': 'tool_call'}])]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'погода сегодня в Москве'}, 'id': '76ebf878-ca39-47a7-a7c9-b20cdcfbba2e', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://world-weather.ru/pogoda/russia/moscow/\', \'content\': \'Погода в Москве сегодня - точный прогноз погоды в Москве на завтра, сейчас (Московская о

In [31]:
messages = [HumanMessage(content="Ну и где теплее?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'погода сегодня теплее'}}, 'functions_state_id': '05e2925d-1a33-4c98-b86b-7aff6801a5c8'}, response_metadata={'token_usage': {'prompt_tokens': 1677, 'completion_tokens': 36, 'total_tokens': 1713}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-baaadb09-94d1-4776-b609-ceba7ac82754-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'погода сегодня теплее'}, 'id': '7fc196ae-963a-4b53-9a47-1c0c32f81372', 'type': 'tool_call'}])]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'погода сегодня теплее'}, 'id': '7fc196ae-963a-4b53-9a47-1c0c32f81372', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content="[{'url': 'https://weather.rambler.ru/', 'content': 'Погода в городах России и мира. Точный и подробный прогноз на сегодня, завтра, выходные, неделю, 14 дней или месяц. ...

In [32]:
messages = [HumanMessage(content="Ну и где теплее?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': {'query': 'where is it warmer'}}, 'functions_state_id': '1dc41216-b612-4d5c-bd65-013175187e3e'}, response_metadata={'token_usage': {'prompt_tokens': 193, 'completion_tokens': 36, 'total_tokens': 229}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-67d1de35-c0f8-44fb-b7ee-d381f2293374-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'where is it warmer'}, 'id': '71e0f969-4b84-4d1c-af72-1261f7abcdf5', 'type': 'tool_call'}])]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'where is it warmer'}, 'id': '71e0f969-4b84-4d1c-af72-1261f7abcdf5', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.climate.gov/news-features/understanding-climate/us-climate-outlook-january-2025\', \'content\': \'U.S. climate outlook for January 2025 | NOAA Climate.gov U.

## Streaming tokens

In [35]:
!pip install langgraph.checkpoint.sqlite

In [52]:
import asyncio
from contextlib import AsyncExitStack
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver

stack = AsyncExitStack()
memory = await stack.enter_async_context(AsyncSqliteSaver.from_conn_string(":memory:"))

abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [56]:
messages = [HumanMessage(content="Какая погода в Санкт-Петербурге?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'погода в Санкт-Петербурге'}, 'id': '7e6aac7f-cea2-4566-9003-929ea9ab5612', 'type': 'tool_call'}
Back to the model!
